# Problem 1

### Decision Variables:
### \$ x_{p,o} \$: amount of raw input p used to produce output o, where o can be Jet A or 100LL.

### Objective Function:
### Minimize the total cost of raw inputs and processing
### Minimize \$ \sum_{p \in P} \sum_{o \in O} (c_{p} + \alpha_{treat} + \alpha_{prod, o} + \alpha_{blend}) \cdot x_{p,o} \$

### Constraints:
### 1. Supply constraint for each raw input p:
### \$ \sum_{o \in O} x_{p,o} \leq u_{p} \$
### 2. Demand Constraints for outputs (Jet A and 100LL):
### \$ \sum_{p \in P} x_{p,Jet A} \geq D_{1} \$
### \$ \sum_{p \in P} x_{p,100LL} \geq D_{2} \$
### 3. Quality Constraints for each requirement r:
### \$ \sum_{p \in P} a_{pr} \cdot x_{p,o} \geq b_{r} \cdot D_{o} \$ for each output o

In [28]:
include("/Users/riyakore/Desktop/CS/CS 524/data_definitions.jl")
using .AvGasData
using JuMP, Cbc

In [29]:
println(AvGasData.inputs)

1:17


In [30]:
m = Model(Cbc.Optimizer)

@variable(m, x[AvGasData.inputs, AvGasData.outputs] >= 0)
@objective(m, Min, sum(AvGasData.rm_cost[i] * x[i,j] + x[i,j] * (AvGasData.blend_cost + (i <= T ? AvGasData.treat_cost : 0) + (j == 1 ? AvGasData.jetA_prod_cost : AvGasData.onehunLL_prod_cost)) for i in AvGasData.inputs, j in AvGasData.outputs))

for i in AvGasData.inputs
    @constraint(m, sum(x[i,j] for j in AvGasData.outputs) <= AvGasData.avail[i])
end


for j in AvGasData.outputs
    @constraint(m, sum(x[i,j] for i in AvGasData.inputs) >= AvGasData.demand[j])
end


for j in AvGasData.outputs
    @constraint(m, sum(x[i,j] * AvGasData.req1_per[i] for i in AvGasData.inputs) / sum(x[i,j] for i in AvGasData.inputs) <= AvGasData.req1_lim)
    @constraint(m, sum(x[i,j] * AvGasData.req2_per[i] for i in AvGasData.inputs) / sum(x[i,j] for i in AvGasData.inputs) <= AvGasData.req2_lim)
end

optimize!(m)
println("Total Cost: ", objective_value(m))
for i in inputs, j in outputs
    println("x[", i, ",", j, "] = ", getvalue(x[i,j]))
end

LoadError: Constraints of type MathOptInterface.ScalarNonlinearFunction-in-MathOptInterface.LessThan{Float64} are not supported by the solver.

If you expected the solver to support your problem, you may have an error in your formulation. Otherwise, consider using a different solver.

The list of available solvers, along with the problem types they support, is available at https://jump.dev/JuMP.jl/stable/installation/#Supported-solvers.